Convolution Neural Networks (CNNs) are a type of deep learning neural network architecture specifically designed for processing grid like data, as such as an image and videos. CNNs have revolutionized the field of computer vision and are widely used for various tasks, including image classification, object detection, facial recognition, and image generation.

In [ ]:
!pip install keras
!pip install tensorflow
!pip install tqdm
!pip install keras_preprocessing

In [ ]:
import os
import pandas as pd
import numpy as np
from keras.utils import to_categorical  # converts a class vector to a binary-class matrix (one-hot-encoding)
from keras_preprocessing.image import load_img # Loads an image from a file and returns it as a PIL image
from keras.models import Sequential # Creates a linear stack of layers for the neural network
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import matplotlib.pyplot as plt

In [ ]:
TRAIN_DIR = r'C:\Users\ishha\OneDrive\Documents\images\train'
TEST_DIR = r'C:\Users\ishha\OneDrive\Documents\images\validation'

In [ ]:
def createdataframe(dir):
    if not os.path.exists(dir):
        raise FileNotFoundError(f"The directory {dir} does not exist.")
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        label_path = os.path.join(dir, label)
        if os.path.isdir(label_path):
            for imagename in os.listdir(label_path):
                image_paths.append(os.path.join(label_path, imagename))
                labels.append(label)
            print(label, "completed")
    return image_paths, labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
train.head()

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
test.head()

In [ ]:
from tqdm import tqdm
def extract_features(images):
    features = []
    for image in tqdm(images):
      img = load_img(image, color_mode='grayscale')
      img = np.array(img)
      features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [ ]:
train_features = extract_features(train['image'])

In [ ]:
test_features = extract_features(test['image'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
model = Sequential()
# Convolution Layers
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.Dropout(0.4)

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.Dropout(0.4)

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.Dropout(0.4)

model.add(Flatten())
# Fully Connected Layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.complie(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x= x_train, y= y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json", "w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
with open("emotiondetector.json", "r") as json_file:
  model_json = json_file.read()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [ ]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [ ]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0